<a href="https://colab.research.google.com/github/alansenairj/chatbot_watson/blob/main/watson_api_share.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# bibliotecas auxiliares e importações de suporte

#biblioteca para executar api flask no colab
!pip install flask-ngrok

# biblioteca para entrar dados mascarados
from getpass import getpass

# Configuração de usuário e senha do email smtp
u = getpass('User: ')
g = getpass('Password: ')

# módulos necessários do flask
from flask import Flask, request, jsonify

# módulos para crawler das informações
import requests
from bs4 import BeautifulSoup

# módulos para envio de e-mail
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# criar app Flask
app = Flask(__name__, static_url_path='')

# apenas para executar no colab
from flask_ngrok import run_with_ngrok
run_with_ngrok(app)

# mapeamento das URLs dos cursos para crawler
curso_urls = {
    'IA':'https://www.impacta.edu.br/mba/artificial-intelligence',
    'Big Data':'https://www.impacta.edu.br/mba/business-intelligence-e-analytics'
}

# GET raiz para teste
@app.route('/')
def root():        
    return "OK"

# endpoint do webhook - POST
@app.route('/watsonwebhook',  methods=['POST'])
def watsonwebhook():
  # retornar dados enviados no formato JSON
  req = request.get_json()

  # tipo da task enviada: info ou email
  task = req['task']
  if task =='info':

    # nome do curso enviado
    curso = req['curso']

    #crawler para pegar informações
    r = requests.get(curso_urls[curso]).text
    info = BeautifulSoup(r, 'html.parser')(class_='background-base-site')[0].text

    #retornar informações no campo info, no formato JSON
    return jsonify({'info':info})
  elif task == 'email':
    try:
      # configuração e envio de email
      message = MIMEMultipart()
      message['From'] = u
      message['To'] = req['email']
      message['Subject'] = 'Confirmação pré-matrícula'
      texto_mensagem = f'Olá! Sua pré-matrícula no curso de {req["curso"]} para o {req["semestre"]} semestre está confirmada'
      message.attach(MIMEText(texto_mensagem, 'plain'))
      
      session = smtplib.SMTP('smtp.gmail.com', 587) 
      session.starttls()
      session.login(u, g)
      text = message.as_string()
      session.sendmail(u, req['email'], text)
      session.quit()
      # retorno de sucesso no formato JSON
      return jsonify({'status':'ok'})
    except Exception as ex:
      print(ex)
      # retorno de erro no formato JSON
      return jsonify({'status':'erro'})
  
# executar app flask
if __name__ == '__main__':
  app.run()




User: ··········
Password: ··········
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://08e76330b2cb.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [30/Jul/2021 22:11:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Jul/2021 22:11:54] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [30/Jul/2021 22:12:17] "POST /watsonwebhook HTTP/1.1" 200 -
127.0.0.1 - - [30/Jul/2021 22:17:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Jul/2021 22:47:37] "POST /watsonwebhook HTTP/1.1" 200 -
127.0.0.1 - - [30/Jul/2021 22:52:16] "POST /watsonwebhook HTTP/1.1" 200 -
